selected fila on origin vy fits, xch slider

In [ ]:
import os
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
from ipywidgets import widgets
from IPython.display import display
from astropy.io import fits
from astropy.wcs import WCS
from datetime import datetime

from astropy.io import fits
import plotly.express as px
import pandas as pd

def read_structure_file(file_path):
    """读取CSV文件并解析数据，以便可视化"""
    df = pd.read_csv(file_path)
    structures = {}
    for _, row in df.iterrows():
        x_coord = int(row['x channel'])  # 确保x channel在CSV中正确命名
        ymin = int(row['ymin'])
        ymax = int(row['ymax'])
        vmin = int(row['vmin'])
        vmax = int(row['vmax'])
        length = float(row['length'])  # 确保'length'列存在且为数值类型
        binary_class = int(row['class'])  # 确保'class'列存在且为整数类型

        # # 只添加长度大于15的结构
        # if binary_class == 1: 
        #     if x_coord not in structures:
        #         structures[x_coord] = []
        #     # 将结构存储为元组，方便后续处理
        if x_coord not in structures:
            structures[x_coord] = []
        structures[x_coord].append((ymin, ymax, vmin, vmax, length))

    return structures


def load_fits_slice_with_structures(x_ch, fits_path, structures):
    """Load a specific x-channel slice from a FITS file, display it using Plotly, and overlay structure ranges."""
    with fits.open(fits_path) as hdul:
        data = hdul[0].data
        wcs = WCS(hdul[0].header)
        data_slice = data[:, :, x_ch].T

    fig = px.imshow(data_slice, origin='lower', labels={'color': 'Intensity'},
                    color_continuous_scale='gray', aspect='equal')
    fig.update_layout(title=f"FITS Slice at X Channel: {x_ch}")

    # Overlay structures from CSV if available for this x channel
    if x_ch in structures:
        for ymin, ymax, vmin, vmax, length in structures[x_ch]:
            # Add a rectangle for each structure
            fig.add_shape(type="rect",
                          x0=vmin, y0=ymin, x1=vmax, y1=ymax,
                          line=dict(color="red"),
                          fillcolor="rgba(0,0,0,0)")

    return fig

def analyze_structures(file_path, fits_path):
    structures = read_structure_file(file_path)
    x_channels = sorted(structures.keys())

    x_slider = widgets.IntSlider(
        value=x_channels[0],
        min=min(x_channels),
        max=max(x_channels),
        step=1,
        description='X Channel:',
        continuous_update=False
    )
    
    plot_container = widgets.Output()
    
    def update_plot(change):
        x_ch = change.new
        with plot_container:
            plot_container.clear_output(wait=True)
            fig = load_fits_slice_with_structures(x_ch, fits_path, structures)
            fig.show()
    
    x_slider.observe(update_plot, names='value')
    display(x_slider, plot_container)

# Set the paths
csv_path = '/Users/naoj306/Desktop/find_data/aquila_12/location/nonan_diff_1-99_3-3/location.csv'
fits_path = "/Users/naoj306/Desktop/DB/SFP/AquilaRift_12CO_21.7arcsec_vel0.1_sph_v1.0.fits"
analyze_structures(csv_path, fits_path)


selected fila on origin vy fits, all xch image

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import pandas as pd
import os

def read_structure_file(file_path):
    """从CSV文件读取并解析数据"""
    df = pd.read_csv(file_path)
    structures = {}
    for _, row in df.iterrows():
        x_coord = int(row['x channel'])
        ymin = int(row['ymin'])
        ymax = int(row['ymax'])
        vmin = int(row['vmin'])
        vmax = int(row['vmax'])
        length = float(row['length'])
        binary_class = int(row['class'])
        if x_coord not in structures:
            structures[x_coord] = []
        # 存储结构及其长度和二分分类
        structures[x_coord].append((ymin, ymax, vmin, vmax, length, binary_class))
    return structures

def save_fits_slice_with_structures(x_ch, fits_path, structures, output_directory):
    """处理FITS切片，添加结构，保存为PNG"""
    with fits.open(fits_path) as hdul:
        data = hdul[0].data
        wcs = WCS(hdul[0].header)
        data_slice = data[:, :, x_ch].T

    fig, ax = plt.subplots()
    ax.imshow(data_slice, origin='lower', cmap='gray', aspect='auto')

    # 如果此x通道有结构，绘制它们
    if x_ch in structures:
        for ymin, ymax, vmin, vmax, length, binary_class in structures[x_ch]:
            # 根据长度决定颜色
            color = 'yellow' if length >= 15 else 'red'
            rect = plt.Rectangle((vmin, ymin), vmax-vmin, ymax-ymin, edgecolor=color, facecolor='none')
            ax.add_patch(rect)

    plt.title(f'FITS Slice and Structures at X Channel: {x_ch}')
    if not os.path.exists(output_directory):
        os.makedirs(output_directory)
    output_path = os.path.join(output_directory, f'x_ch_{x_ch}.png')
    plt.savefig(output_path)
    plt.close()

def process_all_channels(csv_path, fits_path, output_directory):
    structures = read_structure_file(csv_path)
    with fits.open(fits_path) as hdul:
        max_x = hdul[0].data.shape[2]  # Assume the third dimension is X

    for x_ch in range(max_x):
        if x_ch in structures:  # 只处理有结构的x通道
            save_fits_slice_with_structures(x_ch, fits_path, structures, output_directory)


# 设置路径
csv_path = '/Users/naoj306/Desktop/find/location/nonan_diff_1-99_3-3/location.csv'
fits_path = "/Users/naoj306/Desktop/DB/SFP/AquilaRift_12CO_21.7arcsec_vel0.1_sph_v1.0.fits"
output_directory = '/Users/naoj306/Desktop/find/images/1/'
process_all_channels(csv_path, fits_path, output_directory)


plot all fila (x, ymin, ymax) on integrated xy
(too big data, pass)

In [ ]:
# from astropy.io import fits
# from astropy.wcs import WCS
# import numpy as np
# import matplotlib.pyplot as plt
# import pandas as pd
# 
# def generate_and_save_summed_velocity_slices_with_structures(fits_name, output_path, csv_path):
#     with fits.open(fits_name) as hdul:
#         data = hdul[0].data
#         header = hdul[0].header
#         wcs = WCS(header).dropaxis(2)
# 
#     # Check data integrity
#     if np.all(data == 0):
#         raise ValueError("Data contains only zeros.")
#     if np.isnan(data).any():
#         print("Warning: Data contains NaNs, which will be ignored in the sum.")
# 
#     # Sum the data across all velocity channels
#     summed_data = np.nansum(data, axis=0)  # use nansum to ignore NaNs
# 
#     # Determine the dynamic range based on percentiles to handle extreme values
#     vmin = np.percentile(summed_data, 1)   # Adjust this to include more of the data's lower range
#     vmax = np.percentile(summed_data, 99)  # Adjust this to include more of the data's upper range
#     print(f"Using vmin={vmin} and vmax={vmax} for visualization.")
# 
#     df = pd.read_csv(csv_path)
#     fig, ax = plt.subplots(1, 1, figsize=(8, 6))
#     im = ax.imshow(summed_data, origin='lower', cmap='winter', aspect='auto', vmin=vmin, vmax=vmax)
#     ax.set_title("Integrated over all velocities")
# 
#     for _, row in df.iterrows():
#         x_coord = int(row['x channel'])
#         ymin = int(row['ymin'])
#         ymax = int(row['ymax'])
#         ax.scatter([x_coord, x_coord], [ymin, ymax], color='yellow', s=1)
# 
#     ra_axis = np.linspace(0, wcs.pixel_shape[0] - 1, num=5)
#     dec_axis = np.linspace(0, wcs.pixel_shape[1] - 1, num=5)
#     ra_pix, dec_pix = np.meshgrid(ra_axis, dec_axis)
#     ra_deg, dec_deg = wcs.all_pix2world(ra_pix, dec_pix, 0)
#     ax.set_xticks(ra_axis)
#     ax.set_xticklabels([f"{ra:.2f}" for ra in ra_deg[0]])
#     ax.set_yticks(dec_axis)
#     ax.set_yticklabels([f"{dec:.2f}" for dec in dec_deg[:, 0]])
# 
#     ax.set_xlabel("RA (degrees)")
#     ax.set_ylabel("Dec (degrees)")
#     cbar = plt.colorbar(im, ax=ax, pad=0.1)
#     cbar.set_label('Integrated Temperature (K)', rotation=270, labelpad=15)
#     plt.savefig(f"{output_path}/on-xy-map.png")
#     plt.close()
# 
# fits_name = "/Users/naoj306/Desktop/DB/SFP/AquilaRift_12CO_21.7arcsec_vel0.1_sph_v1.0.fits"
# csv_path = '/Users/naoj306/Desktop/find/aquila_12/location/nonan_diff_1-99_3-3/location.csv'
# output_path = "/Users/naoj306/Desktop/find/aquila_12/location/nonan_diff_1-99_3-3/" 
# generate_and_save_summed_velocity_slices_with_structures(fits_name, output_path, csv_path)


plot class=1 (len> ?) (x, ymin, ymax) on integrated xy

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def generate_and_save_summed_velocity_slices_with_structures(fits_name, output_path, csv_path):
    """
    Generate an image by summing slices over all velocity channels, overlay structural data from a CSV file
    for entries where class=1, as lines connecting (x, ymin) to (x, ymax), and save it to a specified path.
    """
    with fits.open(fits_name) as hdul:
        data = hdul[0].data.astype(np.float32)  # Ensure data type is float for handling NaNs
        header = hdul[0].header
        wcs = WCS(header).dropaxis(2)  # Drop the velocity dimension

    if np.isnan(data).any():
        print("Data contains NaNs, filling with zeros.")
        data = np.nan_to_num(data)  # Replace NaNs with zero if present

    # Sum the data across all velocity channels
    summed_data = np.sum(data, axis=0)

    print("Summed Data Min:", np.min(summed_data))
    print("Summed Data Max:", np.max(summed_data))

    # Check if the summed data are properly computed
    if np.all(summed_data == 0):
        raise ValueError("Data contains only zeros.")
    if np.isnan(summed_data).any():
        print("Warning: Summed data contains NaNs, which will be ignored in the visualization.")

    # Determine the dynamic range based on percentiles to handle extreme values
    vmin = np.percentile(summed_data, 1)   # Adjust this to include more of the data's lower range
    vmax = np.percentile(summed_data, 99)  # Adjust this to include more of the data's upper range
    print(f"Using vmin={vmin} and vmax={vmax} for visualization.")
    
    df = pd.read_csv(csv_path)
    df = df[df['class'] == 1]  # Filter to include only rows where class equals 1

    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    im = ax.imshow(summed_data, origin='lower', cmap='winter', aspect='auto', vmin=vmin, vmax=vmax)
    ax.set_title("Integrated over all velocities")

    for _, row in df.iterrows():
        x_coord = int(row['x channel'])
        ymin = int(row['ymin'])
        ymax = int(row['ymax'])
        ax.scatter([x_coord, x_coord], [ymin, ymax], color='yellow', s=0.5)

    # Configure axes with WCS coordinates
    ra_axis = np.linspace(0, wcs.pixel_shape[0] - 1, num=5)
    dec_axis = np.linspace(0, wcs.pixel_shape[1] - 1, num=5)
    ra_pix, dec_pix = np.meshgrid(ra_axis, dec_axis)
    ra_deg, dec_deg = wcs.all_pix2world(ra_pix, dec_pix, 0)
    ax.set_xticks(ra_axis)
    ax.set_xticklabels([f"{ra:.2f}" for ra in ra_deg[0]])
    ax.set_yticks(dec_axis)
    ax.set_yticklabels([f"{dec:.2f}" for dec in dec_deg[:, 0]])

    ax.set_xlabel("RA (degrees)")
    ax.set_ylabel("Dec (degrees)")

    cbar = plt.colorbar(im, ax=ax, pad=0.1)
    cbar.set_label('Integrated Temperature (K)', rotation=270, labelpad=15)

    plt.savefig(f"{output_path}/on-xy-map-30.png")
    plt.close()

fits_name = "/Users/naoj306/Desktop/DB/SFP/AquilaRift_12CO_21.7arcsec_vel0.1_sph_v1.0.fits"
csv_path = '/Users/naoj306/Desktop/find/aquila_12/location/nonan_diff_1-99_3-3/location-30.csv'
output_path = "/Users/naoj306/Desktop/find/aquila_12/location/nonan_diff_1-99_3-3/"
generate_and_save_summed_velocity_slices_with_structures(fits_name, output_path, csv_path)


plot class=1 (x, ymin, ymax) on integrated xy, (x-5, x+5) have neighbor only

In [ ]:
from astropy.io import fits
from astropy.wcs import WCS
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

def generate_and_save_summed_velocity_slices_with_structures(fits_name, output_path, csv_path):
    """
    Generate an image by summing slices over all velocity channels, overlay structural data from a CSV file
    for entries where class=1, as lines connecting (x, ymin) to (x, ymax), and save it to a specified path.
    """
    with fits.open(fits_name) as hdul:
        data = hdul[0].data.astype(np.float32)  # Ensure data type is float for handling NaNs
        header = hdul[0].header
        wcs = WCS(header).dropaxis(2)  # Drop the velocity dimension

    if np.isnan(data).any():
        print("Data contains NaNs, filling with zeros.")
        data = np.nan_to_num(data)  # Replace NaNs with zero if present

    # Sum the data across all velocity channels
    summed_data = np.sum(data, axis=0)

    print("Summed Data Min:", np.min(summed_data))
    print("Summed Data Max:", np.max(summed_data))

    # Check if the summed data are properly computed
    if np.all(summed_data == 0):
        raise ValueError("Data contains only zeros.")
    if np.isnan(summed_data).any():
        print("Warning: Summed data contains NaNs, which will be ignored in the visualization.")

    # Determine the dynamic range based on percentiles to handle extreme values
    vmin = np.percentile(summed_data, 1)   # Adjust this to include more of the data's lower range
    vmax = np.percentile(summed_data, 99)  # Adjust this to include more of the data's upper range
    print(f"Using vmin={vmin} and vmax={vmax} for visualization.")
    
    df = pd.read_csv(csv_path)
    df = df[df['class'] == 1]
    df.sort_values('x channel', inplace=True)

    # 使用相同的代码定义 has_adjacent 函数
    def has_adjacent(row, df):
        x_center = int(row['x channel'])
        x_range = range(x_center - 5, x_center + 6)
        ymin = row['ymin']
        ymax = row['ymax']
        adjacent_df = df[df['x channel'].isin(x_range)]
        for _, adj_row in adjacent_df.iterrows():
            if adj_row['x channel'] != x_center and (adj_row['ymin'] <= ymax and adj_row['ymax'] >= ymin):
                return True
        return False

    df['has_adjacent'] = df.apply(lambda row: has_adjacent(row, df), axis=1)
    df_adjacent = df[df['has_adjacent'] == True]

    fig, ax = plt.subplots(1, 1, figsize=(8, 6))
    im = ax.imshow(summed_data, origin='lower', cmap='winter', aspect='auto', vmin=vmin, vmax=vmax)
    ax.set_title("Integrated over all velocities with adjacent structures")

    for _, row in df_adjacent.iterrows():
        x_coord = [row['x channel'], row['x channel']]
        y_coords = [row['ymin'], row['ymax']]
        ax.scatter(x_coord, y_coords, color='yellow', s=0.5)
        
    # Configure axes with WCS coordinates
    ra_axis = np.linspace(0, wcs.pixel_shape[0] - 1, num=5)
    dec_axis = np.linspace(0, wcs.pixel_shape[1] - 1, num=5)
    ra_pix, dec_pix = np.meshgrid(ra_axis, dec_axis)
    ra_deg, dec_deg = wcs.all_pix2world(ra_pix, dec_pix, 0)
    ax.set_xticks(ra_axis)
    ax.set_xticklabels([f"{ra:.2f}" for ra in ra_deg[0]])
    ax.set_yticks(dec_axis)
    ax.set_yticklabels([f"{dec:.2f}" for dec in dec_deg[:, 0]])
    ax.set_xlabel("RA (degrees)")
    ax.set_ylabel("Dec (degrees)")
    cbar = plt.colorbar(im, ax=ax, pad=0.1)
    cbar.set_label('Integrated Temperature (K)', rotation=270, labelpad=15)

    plt.savefig(f"{output_path}/on-xy-map-30-cluster.png", dpi=100)
    plt.close()


fits_name = "/Users/naoj306/Desktop/DB/SFP/AquilaRift_12CO_21.7arcsec_vel0.1_sph_v1.0.fits"
csv_path = '/Users/naoj306/Desktop/find/aquila_12/location/nonan_diff_1-99_3-3/location-30.csv'
output_path = "/Users/naoj306/Desktop/find/aquila_12/location/nonan_diff_1-99_3-3/"
generate_and_save_summed_velocity_slices_with_structures(fits_name, output_path, csv_path)
